In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

FOLDER_PATH = Path(r'.\test_data\time_sheet2')
excel_files = list(FOLDER_PATH.rglob('*.xlsx'))
data_frames = {}
for xls_file in excel_files:
    data = pd.read_excel(xls_file, sheet_name=None, header=None)
    data_frames[xls_file.name] = data

def reshape_data_frame(df:pd.DataFrame, file_name:str = '', sheet_name:str ='') -> pd.DataFrame:
    location = df.iloc[0,0]
    disciplines = df.iloc[0,1]
    user = df.iloc[1,0]
    user_name = df.iloc[1,1]
    year = df.iloc[2,0]
    month = df.iloc[2,1]
    payment_month = datetime(year=year, month=month,day=1)

    updated_df = df.drop([0,1,2,3])

    updated_df.columns = updated_df.iloc[0]
    updated_df = updated_df.drop([4])

    updated_df['Project Descreption '] = updated_df['Project Descreption '].fillna('')
    updated_df['Package/Zone/Building'] = updated_df['Package/Zone/Building'].fillna('')

    updated_df = updated_df[~updated_df['Project Code'].isna()]
    updated_df = updated_df.fillna(0.0)
    updated_df.loc[:,'row_total'] = updated_df.sum(numeric_only=True, axis=1)

    updated_df['user'] = user
    updated_df['user_name'] = user_name
    updated_df['payment_month'] = payment_month
    updated_df['location'] = location
    updated_df['disciplines'] = disciplines



    updated_df['file_name'] = file_name
    updated_df['sheet_name'] = sheet_name
    return updated_df

data_frames_list = []
for file_name, file_data in data_frames.items():
    for sheet_name, sheet_data in file_data.items():
        if len(sheet_data) > 0:
            data_frames_list.append(reshape_data_frame(sheet_data,file_name,sheet_name).loc[:,['Project Code', 'row_total','location','file_name','payment_month','user', 'user_name']])

final_df = pd.concat(data_frames_list)
final_df.loc[final_df['Project Code'].isin(['C1400','C900']) ]['row_total'].sum()

table = pd.pivot_table(final_df,values='row_total',index=['Project Code'],columns=['payment_month'],aggfunc='sum')
table.fillna(0.0)
table.to_excel('problems.xlsx')

final_df['row_total'].sum()
